## Gradio Pets

In [60]:
# Install FastAI, Gradio, HuggingFace Hub
!pip install fastai gradio huggingface_hub --quiet

##Train the model

In [26]:
# Use Gradio

In [69]:
from fastai.vision.all import *

# Download dataset
path = untar_data(URLs.PETS)
files = get_image_files(path/'images')

# Define DataLoaders
dls = ImageDataLoaders.from_name_re(
    path, files, pat=r'(.+)_\d+.jpg',
    item_tfms=Resize(460),
    batch_tfms=aug_transforms(size=224, min_scale=0.75)
)

# Create learner
learn = vision_learner(dls, models.resnet50, metrics=accuracy)

# Train
learn.fine_tune(1)


epoch,train_loss,valid_loss,accuracy,time
0,1.423045,0.377925,0.885656,01:09


epoch,train_loss,valid_loss,accuracy,time
0,0.426171,0.284209,0.918133,01:17


In [74]:
# Save **weights only** safely
learn.save('model_weights')

Path('/root/.fastai/data/oxford-iiit-pet/models/model_weights.pth')

In [49]:
# Upload example images
from google.colab import files
files.upload()  # select your images

Saving pomerian.jpg to pomerian (1).jpg
Saving brtish.jpg to brtish (1).jpg
Saving birman.jpg to birman (1).jpg


{'pomerian (1).jpg': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00\x84\x00\t\x06\x07\x13\x13\x12\x15\x13\x13\x12\x16\x15\x15\x17\x17\x15\x16\x17\x16\x15\x17\x15\x18\x18\x17\x17\x18\x17\x17\x18\x15\x18\x15\x17\x18\x1d( \x18\x1a%\x1d\x18\x15!1!%)+...\x17\x1f383-7(-.+\x01\n\n\n\x0e\r\x0e\x1a\x10\x10\x1a+%\x1e\x1d--------------/-----------+---------------------5-\xff\xc0\x00\x11\x08\x00\xe1\x00\xe1\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1c\x00\x00\x02\x03\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x04\x05\x02\x03\x06\x01\x00\x07\x08\xff\xc4\x00E\x10\x00\x01\x03\x02\x03\x05\x04\x07\x04\x08\x04\x06\x03\x00\x00\x00\x01\x00\x02\x11\x03!\x04\x121\x05AQaq\x13"\x81\x91\x06\x142\xa1\xb1\xc1\xf0Rr\xd1\xf1\x07#3Bs\xb2\xc2\xe14Sb\x83\x15C\x82\x93\xb3\xd2\x16c\xa2\xff\xc4\x00\x18\x01\x00\x03\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\xff\xc4\x00!\x11\x01\x01\x00\x02\x03\x00\x02\x03\x01\x01\x00\x00\x00\x00\x00\x00

In [84]:
import gradio as gr
from fastai.vision.all import *

# -----------------------
# Load exported FastAI learner
# -----------------------
learn = load_learner('model.pkl')  # export.pkl contains full learner (weights + transforms + dls)

# -----------------------
# Labels from learner
# -----------------------
labels = learn.dls.vocab

# -----------------------
# Prediction function
# -----------------------
def predict(img):
    img = PILImage.create(img)
    pred, pred_idx, probs = learn.predict(img)
    return {labels[i]: float(probs[i]) for i in range(len(labels))}

# -----------------------
# Gradio interface
# -----------------------
examples = ['birman.jpg', 'british.jpg', 'pomerian.jpg']  # make sure these exist in repo

demo = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil"),
    outputs=gr.Label(num_top_classes=3),
    examples=examples,
    title="Pet Classifier",
    description="Upload a pet image to classify."
)

# -----------------------
# Launch app
# -----------------------
demo.launch()


/usr/local/lib/python3.12/dist-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5c156fad780069d85d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [85]:
%%writefile /content/requirements.txt
fastai
gradio
torch
scikit-image

Overwriting /content/requirements.txt


In [88]:
from huggingface_hub import notebook_login, HfApi, Repository

notebook_login()  # paste your HF token

In [92]:
from huggingface_hub import HfApi, Repository

SPACE_ID = "sks01dev/Dogs_and_Cats_Classifier"  # your space
LOCAL_DIR = "/content/hf_space"


In [93]:
!rm -rf {LOCAL_DIR}  # remove old clone
repo = Repository(local_dir=LOCAL_DIR, clone_from=SPACE_ID, repo_type="space", use_auth_token=True)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/spaces/sks01dev/Dogs_and_Cats_Classifier into local empty directory.


Download file model_weights.pth:   0%|          | 8.00k/294M [00:00<?, ?B/s]

Download file birman.jpg: 100%|##########| 4.21k/4.21k [00:00<?, ?B/s]

Download file pomerian.jpg: 100%|##########| 7.54k/7.54k [00:00<?, ?B/s]

Download file british.jpg: 100%|##########| 7.88k/7.88k [00:00<?, ?B/s]

...

Clean file birman.jpg:  24%|##3       | 1.00k/4.21k [00:00<?, ?B/s]

Clean file pomerian.jpg:  13%|#3        | 1.00k/7.54k [00:00<?, ?B/s]

Download file model.pkl:   0%|          | 1.39k/98.3M [00:00<?, ?B/s]

Clean file model.pkl:   0%|          | 1.00k/98.3M [00:00<?, ?B/s]

Clean file model_weights.pth:   0%|          | 1.00k/294M [00:00<?, ?B/s]

In [94]:
!cp /content/app.py {LOCAL_DIR}/
!cp /content/model.pkl {LOCAL_DIR}/
!cp birman.jpg british.jpg pomerian.jpg {LOCAL_DIR}/
!cp /content/requirements.txt {LOCAL_DIR}/  # make sure fastai + gradio + torch included

In [96]:
!git config --global user.email "sswarnkar0001@gmail.com"
!git config --global user.name "sks01dev"


In [97]:
import os

# Go to repo directory
os.chdir(LOCAL_DIR)

# Remove git history to ensure one clean commit
!rm -rf .git
!git init
!git lfs install
!git lfs track "*.pkl"

# Stage all files
!git add .
!git commit -m "Clean push: working app.py with model and examples"

# Add HF remote again
!git remote add origin https://huggingface.co/spaces/sks01dev/Dogs_and_Cats_Classifier
!git branch -M main
!git push -u origin main --force


hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/hf_space/.git/
Updated git hooks.
Git LFS initialized.
"*.pkl" already supported
[master (root-commit) 5ac4cea] Clean push: working app.py with model and examples
 9 files changed, 104 insertions(+)
 create mode 100644 .gitattributes
 create mode 100644 README.md
 create mode 100644 app.py
 create mode 100644 birman.jpg
 create mode 100644 british.jpg
 create mode 100644 model.pkl
 create mode 100644 model_weights.pth
 create mode 100644 pomerian.jpg
 create mode 10